# Extract Raw Data

In [ ]:
import pandas as pd
import yahoo_fin.stock_info as si
import os

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
SP500_company_data = pd.read_html(url)[0]
SP500_company_data.to_csv("companies_data.csv")

print(SP500_company_data['Symbol'][:10])
print('Total de empresas: ',len(SP500_company_data))

0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
5    ADBE
6     AMD
7     AES
8     AFL
9       A
Name: Symbol, dtype: object
Total de empresas:  503


### Train Data

In [3]:
SP500_train_data = pd.DataFrame()

for company in SP500_company_data['Symbol']:
    try:
        company_data = si.get_data(company, start_date="01/01/2010", end_date="12/31/2023", index_as_date = True, interval="1d")
        SP500_train_data = pd.concat([SP500_train_data, company_data])
    except:
        ...

SP500_train_data.to_csv("stocks_train_data.csv")

In [4]:
SP500_train_data.head()

,open,high,low,close,adjclose,volume,ticker
2010-01-04,69.473244,69.774246,69.122070,69.414719,43.783871,3640265,MMM
2010-01-05,69.230766,69.590302,68.311035,68.979935,43.509636,3405012,MMM
2010-01-06,70.133781,70.735786,69.824417,69.958191,44.126671,6301126,MMM
2010-01-07,69.665550,70.033447,68.662209,70.008362,44.158295,5346240,MMM
2010-01-08,69.974915,70.501671,69.648827,70.501671,44.469471,4073337,MMM


### Test Data

In [5]:
SP500_test_data = pd.DataFrame()

for company in SP500_company_data['Symbol']:
    try:
        company_data = si.get_data(company, start_date="01/01/2024", end_date="01/31/2024", index_as_date = True, interval="1d")
        SP500_test_data = pd.concat([SP500_test_data, company_data])
    except:
        ...

SP500_test_data.to_csv("stocks_test_data.csv")

In [6]:
SP500_test_data.head()

,open,high,low,close,adjclose,volume,ticker
2024-01-02,90.819397,92.525085,90.677261,91.973244,88.892036,3321053,MMM
2024-01-03,91.329430,91.521736,89.297661,90.125420,87.106117,3547575,MMM
2024-01-04,90.367889,91.421402,90.058525,90.443146,87.413200,3319976,MMM
2024-01-05,90.284279,91.546822,89.924751,90.794312,87.752602,1991579,MMM
2024-01-08,90.518394,91.103676,89.958191,91.020065,87.970787,2535042,MMM


### Verificar se as empresas são iguais em ambos 

In [7]:
# verificar se contem o msm numero de empresas
train_ind = SP500_train_data["ticker"].unique()
test_ind = SP500_test_data["ticker"].unique()

diff = False
for ticker in train_ind:
    if ticker not in test_ind:
        diff = True
        break

if diff: print("Datasets contem diferentes empresas")
else: print("Datasets contem as mesmas empresas, sendo o numero total: %d" % len(test_ind))

Datasets contem as mesmas empresas, sendo o numero total: 497


## Vamos decidir quais empresas usar

Vamos escolher os 5 melhores setores e as respetivas 30 melhores empresas com base no <b> crescimento mensal </b>

In [2]:
# carregar os dados
SP500_company_data = pd.read_csv("companies_data.csv", index_col=0)
SP500_test_data = pd.read_csv("stocks_test_data.csv", index_col=0)
SP500_test_data.index = pd.to_datetime(SP500_test_data.index)
SP500_train_data = pd.read_csv("stocks_train_data.csv", index_col=0)
SP500_train_data.index = pd.to_datetime(SP500_train_data.index)

In [8]:
for sector in SP500_company_data["GICS Sector"].unique():
    n_empresas = SP500_company_data[SP500_company_data["GICS Sector"] == sector]["GICS Sector"].count()
    print(f"Total de empresas no setor '{sector}' -> {n_empresas}")

Total de empresas no setor 'Industrials' -> 78
Total de empresas no setor 'Health Care' -> 62
Total de empresas no setor 'Information Technology' -> 69
Total de empresas no setor 'Utilities' -> 31
Total de empresas no setor 'Financials' -> 72
Total de empresas no setor 'Materials' -> 28
Total de empresas no setor 'Consumer Discretionary' -> 50
Total de empresas no setor 'Real Estate' -> 31
Total de empresas no setor 'Communication Services' -> 22
Total de empresas no setor 'Consumer Staples' -> 38
Total de empresas no setor 'Energy' -> 22


In [9]:
def get_growth(df):
    # retorna o crescimento mensal da empresa 
    monthly_growth = 0
    monthly = df['close'].resample('ME').agg(['first', 'last'])
    
    for _, row in monthly.iterrows():
        monthly_growth += ((row['last'] - row['first']) / row['first']) * 100
        
    return monthly_growth/len(monthly)

In [10]:
growth = {}
sectors_growth = {}
for sector in SP500_company_data["GICS Sector"].unique():
    mean_growth_sector = 0
    sector_data = SP500_company_data[SP500_company_data["GICS Sector"] == sector]
    data = {}
    
    for company in sector_data["Symbol"]:
        try:
            stock_growth = get_growth(SP500_train_data[SP500_train_data["ticker"]==company])
            mean_growth_sector += stock_growth
            data[company] = stock_growth
        except: continue
        
    growth[sector] = dict(sorted(data.items(), key=lambda item: item[1], reverse=True)) # ordenar por ordem decrescente
    sectors_growth[sector] = mean_growth_sector/len(data)
# ordenar em forma decrescente
sectors_growth = dict(sorted(sectors_growth.items(), key=lambda item: item[1], reverse=True))

### Identificar os melhores setores e respetivas melhores empresas

Identificamos quais os 5 melhores setores e as respetivas 30 melhores empresas

In [12]:
if not os.path.exists("dataset_by_sector/"):
    os.mkdir("dataset_by_sector/")
    
metadata = pd.DataFrame(columns=['ticker', 'sector', 'model'])
# metadata = metadata.append({'tinker': 'Tinker3', 'sector': 'Sector3', 'model': 'Model3'}, ignore_index=True)
j = 1
for sector,m_growth in sectors_growth.items():
    if j > 5: break
    i = 1
    print(f"{j}) Setor: {sector} (avg monthly growth: {m_growth:.4f}%) - all companies in sector")
    print("    < 30 best companies >")
    for company,data in growth[sector].items():
        if i > 30: break
        print(f"    {company} - avg monthly growth: {data:.4f}%")
        metadata.loc[len(metadata)] = [company,sector.replace(" ","_"),"por_definir"]
        # quando criarmos os modelos vamos associar a cada stock
        i += 1
    j += 1
metadata.to_csv(f"dataset_by_sector/metadata.csv")

1) Setor: Information Technology (avg monthly growth: 1.7003%) - all companies in sector
    < 30 best companies >
    PLTR - avg monthly growth: 5.0461%
    ENPH - avg monthly growth: 4.1417%
    CRWD - avg monthly growth: 3.7929%
    NVDA - avg monthly growth: 3.4466%
    ANET - avg monthly growth: 3.0379%
    NOW - avg monthly growth: 2.8668%
    AVGO - avg monthly growth: 2.8320%
    EPAM - avg monthly growth: 2.7924%
    PANW - avg monthly growth: 2.6617%
    FTNT - avg monthly growth: 2.5234%
    FICO - avg monthly growth: 2.5175%
    MPWR - avg monthly growth: 2.4459%
    SMCI - avg monthly growth: 2.4381%
    CDNS - avg monthly growth: 2.4317%
    AMD - avg monthly growth: 2.1137%
    DELL - avg monthly growth: 2.0834%
    KEYS - avg monthly growth: 2.0398%
    IT - avg monthly growth: 2.0366%
    SNPS - avg monthly growth: 2.0186%
    ZBRA - avg monthly growth: 1.9960%
    NXPI - avg monthly growth: 1.9517%
    LRCX - avg monthly growth: 1.9481%
    INTU - avg monthly growth: 

### Guardar os dados

Vamos agora guardar os dados das 30 melhores empresas por setor

In [13]:
if not os.path.exists("dataset_by_sector/train/"):
    os.mkdir("dataset_by_sector/train/")
    
if not os.path.exists("dataset_by_sector/test/"):
    os.mkdir("dataset_by_sector/test/")    
    
limit_sector = 1

for sector,m_growth in sectors_growth.items():
    if limit_sector > 5: break
    limit_companies = 1
    
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()
    
    for company,data in growth[sector].items():
        if limit_companies > 30: break

        df_train = pd.concat([df_train,SP500_train_data[SP500_train_data["ticker"]==company]])
        df_test = pd.concat([df_test,SP500_test_data[SP500_test_data["ticker"]==company]])
        limit_companies += 1

    df_train.to_csv(f"dataset_by_sector/train/{sector.replace(" ","_")}.csv")
    df_test.to_csv(f"dataset_by_sector/test/{sector.replace(" ","_")}.csv")
    limit_sector += 1


### Remover os datasets iniciais 

Remover os datasets que já não são necessários

In [14]:
del_files = ["companies_data.csv","stocks_train_data.csv","stocks_test_data.csv"]

for csv_file in del_files:
    if os.path.exists(csv_file):
        os.remove(csv_file)
        print(f"Ficheiro '{csv_file}' removido com sucesso.")
    else:
        print(f"Ficheiro '{csv_file}' não encontrado.")

Ficheiro 'companies_data.csv' removido com sucesso.
Ficheiro 'stocks_train_data.csv' removido com sucesso.
Ficheiro 'stocks_test_data.csv' removido com sucesso.


<hr>
tirar ?

# Parte do chico

### Decidir empresas com melhor crescimento

In [34]:
file_path = "stocks_train_data.csv"  
data = pd.read_csv(file_path)

if 'open' in data.columns and 'close' in data.columns and 'ticker' in data.columns:

    data['Percentile Growth'] = ((data['close'] - data['open']) / data['open']) * 100

    unique_companies = data.loc[data.groupby('ticker')['Percentile Growth'].idxmax()]

    sorted_data = unique_companies.sort_values(by='Percentile Growth', ascending=False)

    top_20_companies = sorted_data.head(50)

    print(top_20_companies[['ticker', 'open', 'close', 'Percentile Growth']])

    top_20_companies.to_csv("top_20_companies_1day.csv", index=False)
else:
    print("Data is missing")



        ticker         open        close  Percentile Growth
1222466    PCG     7.900000    13.950000          76.582274
650933    FSLR    27.139999    39.349998          44.988944
571668     EQT     4.920000     7.060000          43.495932
210904    BIIB   253.199997   355.630005          40.454190
1634373    WMB    13.420000    18.290001          36.289127
1444320   TRGP     4.780000     6.390000          33.682000
1316769     RF    14.210000    18.850000          32.653064
736689     HIG    25.980000    33.520000          29.022329
1515807   UBER    15.960000    20.490000          28.383457
1178302    OKE    16.100000    20.490000          27.267076
1543983    UHS    66.669998    84.839996          27.253635
1493506    TDG   242.100006   305.769989          26.299042
1190852   PLTR    25.850000    32.580002          26.034821
456277     DRI    33.730000    42.419998          25.763412
1462883   TSLA     1.266667     1.592667          25.736833
1174780     ON     8.360000    10.510000

In [35]:
file_path = "stocks_train_data.csv"
data = pd.read_csv(file_path)

if 'open' in data.columns and 'close' in data.columns and 'ticker' in data.columns:

    data['Percentual Crescimento Diário'] = ((data['close'] - data['open']) / data['open']) * 100

    media_crescimento = data.groupby('ticker')['Percentual Crescimento Diário'].mean().reset_index()

    media_crescimento.rename(columns={'Percentual Crescimento Diário': 'Média Crescimento Diário'}, inplace=True)

    top_50_empresas = media_crescimento.sort_values(by='Média Crescimento Diário', ascending=False).head(100)

    print(top_50_empresas)

    top_50_empresas.to_csv("top_50_empresas_daily average.csv", index=False)
else:
    print("Data is missing")


    ticker  Média Crescimento Diário
465   VLTO                  0.586928
83     CEG                  0.238511
73    CARR                  0.165847
438    TPL                  0.157252
3     ABNB                  0.119603
..     ...                       ...
220    HII                  0.058211
156    ELV                  0.057973
396    ROL                  0.057695
447     TT                  0.057579
119   CTVA                  0.057425

[100 rows x 2 columns]


In [36]:
file_path = "stocks_train_data.csv"  
data = pd.read_csv(file_path)

if 'open' in data.columns and 'close' in data.columns and 'ticker' in data.columns:
    grouped = data.groupby('ticker')

    summary = grouped.agg(
        First_Open=('open', 'first'),
        Last_Close=('close', 'last')
    ).reset_index()

    summary['Percentual Crescimento'] = ((summary['Last_Close'] - summary['First_Open']) / summary['First_Open']) * 100

    top_100_empresas = summary.sort_values(by='Percentual Crescimento', ascending=False).head(20)

    print(top_100_empresas)

    top_100_empresas.to_csv("top_100_empresas.csv", index=False)
else:
    print("Erro: As colunas necessárias ('open', 'close', 'ticker') estão ausentes nos dados.")


    ticker  First_Open   Last_Close  Percentual Crescimento
445   TSLA    1.266667   248.479996            19516.836476
341   NVDA    0.462750    49.521999            10601.674913
331   NFLX    7.931429   486.880005             6038.616515
43    AVGO    1.830000   111.625000             5999.726633
147   DXCM    2.052500   124.089996             5945.797601
46    AXON    4.460000   258.329987             5692.152116
460    URI    9.920000   573.419983             5680.443332
185   FICO   21.379999  1164.010010             5344.387537
438    TPL   10.146667   524.150024             5065.736186
141    DPZ    8.530000   412.230011             4732.708369
80    CDNS    6.010000   272.369995             4431.946502
347   ODFL    4.548148   202.664993             4355.989259
61    BLDR    3.930000   166.940002             4147.837140
390   REGN   24.240000   878.289978             3523.308524
285   LULU   15.435000   511.290009             3212.536408
193   FTNT    1.788000    58.529999     